In [1]:
from find_source import full_html_and_txt, make_catalog, summary
import numpy as np
import glob
import json

In [2]:
for file in glob.glob('../data/250611_03:56:34/*.fits'): #('../data/multi_track/*.fits'):
    try:
        s = summary(file, plot=False)
        rms = s['rms_val']
        rms_from_peak = s['calc_rms_val']
        sd_mad = s['sd_mad']
        i = file.rindex('/')
        print(f'file: {file[i+1:]}, rms: {round(rms, 5)}, sd_mad: {round(sd_mad, 5)}, rms_from_peak: {round(rms_from_peak, 5)}')
    except Exception as e:
        print(f'Error for {file}: {e}')

file: 1310+323_lr_full.fits, rms: 0.01139, sd_mad: 0.0104, rms_from_peak: 0.02669
file: 1635+381_full.fits, rms: 0.00354, sd_mad: 0.00353, rms_from_peak: 0.00749
file: 1642+398_full.fits, rms: 0.00699, sd_mad: 0.00597, rms_from_peak: 0.01675
file: 3c286_full.fits, rms: 0.00115, sd_mad: 0.00108, rms_from_peak: 0.00295
file: 1512-090_full.fits, rms: 0.00274, sd_mad: 0.00266, rms_from_peak: 0.00719
file: 3c279_lr_full.fits, rms: 0.07091, sd_mad: 0.06342, rms_from_peak: 0.13096
file: 1159+292_full.fits, rms: 0.00524, sd_mad: 0.00445, rms_from_peak: 0.01379
file: mwc349a_full.fits, rms: 0.00776, sd_mad: 0.00762, rms_from_peak: 0.02124
Error for ../data/250611_03:56:34/1743-038_full.fits: "Keyword 'BMAJ' not found."
file: 1310+323_full.fits, rms: 0.00495, sd_mad: 0.00462, rms_from_peak: 0.01331
file: 1555+111_full.fits, rms: 0.00364, sd_mad: 0.00373, rms_from_peak: 0.00733
file: 1751+096_full.fits, rms: 0.00404, sd_mad: 0.00366, rms_from_peak: 0.00988
file: 3c279_full.fits, rms: 0.01686, sd_

In [3]:
%%timeit
full_html_and_txt('../data/250611_03:56:34')

7.64 s ± 952 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
folders = []
for folder in glob.glob('/reduction/karto/SMA/polaris_test/*'):
    print(folder)
    folders.append(folder)

In [ ]:
full_html_and_txt(folders[0])
print(folders[0])
folders = folders[1:]

In [ ]:
'''
Using the exclusion region's rms taken directly from the image,
finds the probability of detecting the inclusion region's maximum flux if there were no source in the inclusion region,
the probability of detecting the exclusion region's maximum flux if there were no source in the exclusion region, and other statistics.

If the external probability is less than 0.001, updates the inclusion region to include a circle around the external peak.

The other statisitcs are the inclusion region's maximum flux in Jy and its coordinates in pixels,
the exclusion region's maximum flux in Jy and its coordinates in pixels, the exclusion region's rms in Jy,
the number of measurements in the inclusion region, the number of measurements in the exclusion region,
the coordinates in pixels of the image's center, and the radii in pixels of the inclusion zones,
the inclusion region's signal to noise ratio, and the external region's signal to noise ratio.

Parameters
----------
fits_file : str
    The path of the FITS file that contains the image.
center : list (optional)
    A list of center coordinates in units of pixels.
    If no center coordinates are given, eventually defaults to ((length of x-axis)/2, (length of y-axis)/2), rounded up.
radius_buffer : float (optional)
    The amount of buffer, in arcsec, to add to the beam FWHM to get the initial search radius.
    If no value is given, defaults to 5 arcsec.
ext_threshold : float (optional)
    The probability that an external peak must be below for it to be considered an external source.
    If no value is given, defaults to 0.001.
rms : float (optional)
    An rms value in Jy.
    If no value is given, eventually defaults to the rms calculated by incl_excl_data.
recursion : bool (optional)
    Whether to use recursion to find significant external peaks, if any.
    If no value is given, defaults to True.

Returns
-------
list
    A list with:
        dict (possibly multiple)
            A dictionary with:
                float
                    The probability of detecting the inclusion region's maximum flux if there were no source in the inclusion region.
                float
                    The probability of detecting the exclusion region's maximum flux if there were no source in the exclusion region.
                float
                    The inclusion region's maximum flux in Jy.
                tuple (int, int)
                    The coordinates in pixels of the inclusion region's maximum flux.
                float
                    The exclusion region's maximum flux in Jy.
                tuple (int, int)
                    The coordinates in pixels of the exclusion region's maximum flux.
                float
                    The exclusion region's rms in Jy.
                float
                    The number of measurements in the inclusion region.
                float
                    The number of measurements in the exclusion region.
                tuple (int, int)
                    The coordinates in pixels of the image's center.
                list
                    A list with:
                        float(s)
                            The radii in pixels of inclusion zones.
                float
                    The inclusion region's signal to noise ratio.
                float
                    The exclusion region's signal to noise ratio.
'''

In [ ]:
'''
Finds statistics of an inclusion region and its complement, the exclusion region.

The inclusion region can be the union of circles or the complement of such a region.

The statistics are the inclusion region's maximum flux in Jy and its coordinates in pixels,
the exclusion region's maximum flux in Jy and its coordinates in pixels, the exclusion region's rms in Jy,
the number of measurements in the inclusion region, the number of measurements in the exclusion region,
the coordinates in pixels of the image's center, and the radii in pixels of the inclusion zones.

Parameters
----------
fits_file : str
    The path of the FITS file that contains the image.
center : list (optional)
    A list of center coordinates in units of pixels.
    If no center coordinates are given, eventually defaults to ((length of x-axis)/2, (length of y-axis)/2), rounded up.
radius_buffer : float (optional)
    The amount of buffer, in arcsec, to add to the beam FWHM to get the initial search radius.
    If no value is given, defaults to 5 arcsec.

Returns
-------
dict
    A dictionary with:
        float
            The inclusion region's maximum flux in Jy.
        tuple (int, int)
            The coordinates in pixels of the inclusion region's maximum flux.
        float
            The exclusion region's maximum flux in Jy.
        tuple (int, int)
            The coordinates in pixels of the exclusion region's maximum flux.
        float
            The exclusion region's rms in Jy.
        float
            The number of measurements in the inclusion region.
        float
            The number of measurements in the exclusion region.
        tuple (int, int)
            The coordinates in pixels of the image's center.
        list
            A list with:
                float(s)
                    The radii in pixels of inclusion zones.
'''